In [ ]:
import pandas as pd
import json
from my_jira import my_jira_reports

import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('data.json', 'r') as read_file:
    data = json.load(read_file)
jr=my_jira_reports(data['server'],'token',data['token'],verify_cert=False)
project,team='AS',data['team']

In [ ]:
jr.issue_raw_info('DRS20211-1496')

### Отчёт по незавершенным задачам

In [ ]:
df=jr.undone_custom_report(project)
jr.color_undone_custom_report(df)

### Обновляем статусы

In [ ]:
#jr.get_task_transitions('MIGRCEH3-127')
#jr.transition_issue('MIGRCEH3-127',121,'Готово')
jr.mass_transition(df)

In [ ]:
df=jr.undone_custom_report(project)
jr.color_undone_custom_report(df)

### Инфо по задаче

In [ ]:
full_info=jr.get_issue('MIGRCEH1-642')
jr.extract_issue_info(full_info)

In [ ]:
full_info['fields']['issuelinks']

### Команда

In [ ]:
jr.show_team(data['team'])

### Worklog

In [ ]:
%%time
start_date='2021-10-01'
wl=jr.worklog_rep(data['team'],start_date=start_date)
wr=wl[wl.log_date>=start_date]

In [ ]:
wr.pivot_table(index=['key','summary'], columns='author', values='hours',aggfunc=sum,margins=True).fillna(0)

### Создание задач

In [ ]:
_path=r'___________.xlsx'
input_data=pd.read_excel(_path,parse_dates=False)
input_data.assignee.unique()
team_dict={'
          }
input_data['assignee_name']=input_data['assignee'].apply(lambda x:team_dict[x])
input_data

In [ ]:
def vtb_issue(project:str,row:dict,sprint=None):
    additional_keys={'customfield_16206': [{'key':'ST-191257'}],
                 'customfield_16201': [{'key':'SYS-358683'}],
                 'customfield_15715': [{'key':'AG-1453383'}],
                     'customfield_11164':str(row['start_date'])[:10],
                     'customfield_11163':str(row['end_date'])[:10],
                     'duedate':str(row['end_date'])[:10],
                 'priority':{'name': 'Medium'},
                 "timetracking": {"originalEstimate": row['estimate'],
                                  "remainingEstimate": row['estimate'],
                                 },
                 "assignee":{'name': row['assignee_name']}
                }
    issue=jr.create_issue(project=project,
                  issue_type='Задача',
                  task_summary=row['task_summary'],
                  task_description='',
                  additional_keys=additional_keys)
    epic=row['epic']
    #return issue,epic
    key=issue.json()['key']
    if sprint is not None:
        jr.issue_to_sprint(sprint, [key])
    
    if row['epic'] is not pd.np.nan:
        jr.issue_to_epic(row['epic'],[key])
        
    return (i,key,row['assignee'])
    

In [ ]:
jr.issue_to_epic('ANALYTREP-1328',['MIGRCEH3-148'])

In [ ]:
jr.issue_to_epic('ANALYTREP-1328',['MIGRCEH3-148'])

In [ ]:
additional_keys={'customfield_16206': [{'key':'ST-191257'}],
                 'customfield_16201': [{'key':'SYS-358683'}],
                 'customfield_15715': [{'key':'AG-1453383'}],
                 'priority':{'name': 'Medium'},
                 "timetracking": {"originalEstimate": "1w 3d",
                                  "remainingEstimate": "1w 3d",
                                 },
                 "assignee":{'name': '____________'}
                }

In [ ]:
t=jr.create_issue(project='______',
                  issue_type='Задача',
                  task_summary='Корневая задача branch_x_abs_hist',
                  task_description='',
                  additional_keys=additional_keys)